In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic, KNNWithMeans, accuracy
from surprise.model_selection import train_test_split, cross_validate


In [2]:
df = pd.read_parquet('dataset_ml.parquet')

In [3]:
df.head(5)

,userId,score,id,title
1,1,4.5,ns2186,latte and the magic waterstone
3,1,5.0,ns3663,"frankenstein’s monster’s monster, frankenstein"
4,1,5.0,as9500,kept woman
6,1,4.5,ns8282,the drowning
7,1,5.0,as6112,salome


In [4]:
# Creo los objetos Reader y Dataset
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['userId','id','score']],reader=reader)

In [5]:
# Separo datos
trainset, testset = train_test_split(data,test_size=30)

In [6]:
# Usamos un modelo de SVD y Validacion cruzada
model = SVD()

In [7]:
#Entrenamos el modelo
model.fit(trainset)

In [8]:
prediccion = model.test(testset)

In [9]:
prediccion[1]

Prediction(uid=33657, iid='ns2941', r_ui=3.5, est=3.1325635336155626, details={'was_impossible': False})

In [10]:
#Prediccion al azar para usuario y película
model.predict(2010,22)

Prediction(uid=2010, iid=22, r_ui=None, est=3.414633016157432, details={'was_impossible': False})

In [11]:
#tomo usuario para hacerle una recomendacion
usuario = 1005
rating = 3.5
df_user = df[(df['userId']==usuario) & (df['score'] >= rating)]
df_user = df_user.reset_index(drop=True)
df_user

,userId,score,id,title
0,1005,4.0,as5663,brittany runs a marathon
1,1005,5.0,as4508,nene raju nene mantri
2,1005,5.0,ns7253,la bamba
3,1005,4.0,as1720,master (tamil) [4k uhd]
4,1005,3.5,as9200,first shot


In [12]:
#Probamos el modelo
usuario = 1005
recomendaciones = []
for item in range(1,len(model.trainset.ir)):
    if not model.trainset.knows_item(item):
        prediccion_score = model.predict(usuario, item).est
        recomendaciones.append((item,prediccion_score))

# ordeno recomendacion por score
sorted_recomendacion = sorted(recomendaciones, key=lambda x: x[1], reverse=True)

# Top 5 de recomendaciones
top_5 = [item[0] for item in sorted_recomendacion[:5]]
top_5

[]

In [13]:
recomendacion = df.iloc[:1000].copy()
recomendacion

,userId,score,id,title
1,1,4.5,ns2186,latte and the magic waterstone
3,1,5.0,ns3663,"frankenstein’s monster’s monster, frankenstein"
4,1,5.0,as9500,kept woman
6,1,4.5,ns8282,the drowning
7,1,5.0,as6112,salome
...,...,...,...,...
1374,20,5.0,as8945,"me, myself, and i"
1376,20,4.0,ns7035,hypersomnia
1377,20,3.0,as9214,dolls
1378,20,4.5,ns6069,a.m.i.


In [14]:
usuario_vistas = df[df['userId']== usuario]
usuario_vistas.shape

(11, 4)

In [15]:
usuario_vistas

,userId,score,id,title
99272,1005,4.0,as5663,brittany runs a marathon
99275,1005,2.0,as380,the world's most breathtaking waterfalls
99276,1005,3.0,as5120,eh janam tumhare lekhe
99277,1005,5.0,as4508,nene raju nene mantri
99279,1005,5.0,ns7253,la bamba
99281,1005,2.5,ns8042,slow west
99282,1005,4.0,as1720,master (tamil) [4k uhd]
99283,1005,3.0,as6032,life itself
99284,1005,3.0,as7766,"barbara stanwyck & sterling hayden in ""crime o..."
99285,1005,2.5,as8635,the best things in the world


In [16]:
# sacamos las vistas ya por el usuario
recomendacion.drop(usuario_vistas.userId,inplace=True)
recomendacion = recomendacion.reset_index()
recomendacion

KeyError: '[1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005] not found in axis'

In [17]:
recomendacion['Estimacion'] = recomendacion['id'].apply(lambda x:model.predict(usuario,x).est)

In [18]:
recomendacion = recomendacion.sort_values('Estimacion',ascending=False)
recomendacion.head()

,userId,score,id,title,Estimacion
39,2,4.0,ns3429,el camino: a breaking bad movie,4.210400
148,6,3.0,ds1035,marvel rising: secret warriors,4.195578
698,12,3.0,ns584,no strings attached,4.108364
929,15,5.0,as6128,factory girl,4.032686
629,11,3.0,ds955,how to fish,4.016423


### **Evaluación del modelo**

In [19]:
accuracy.rmse(prediccion)

RMSE: 1.2128


1.2128453875430423

### Query: get_recomendation 
Consigna: recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función adicional de la API anterior y debe llamarse get_recommendation(titulo: str).

In [37]:
def get_recomendation(title:str):
    movie_id = df[df['title']==title]['id'].iloc[0]
    movie_inner_id = trainset.to_inner_iid(movie_id)
    prediccion = []

    if movie_id is None:
        return 'Esta pelicula no se encuentra, intente con otro titulo'

    for uid in range(trainset.n_users):
        prediccion.append((uid, model.predict(uid,movie_inner_id).est))
    
    prediccion.sort(key=lambda x: x[1], reverse=True)
    respuesta = [items[0] for items in prediccion[:5]]

    respuesta = []
    for item in prediccion[:5]:
        titulos = df[df['id']==item[0]]
        pelicula = titulos['title'].iloc[0]
        respuesta.append(pelicula)
    return {'Recomendación': respuesta}
   


In [38]:
get_recomendation("marvel studios' avengers: infinity war")

{'Recomendación': [46854, 4434, 22335, 19510, 44683]}